# 02 — Квантовые ансамбли (ring / torus / RRG / rewired) → признаки → сравнение с космологией

**Цель:** выполнить сравнение графовой геометрии космологических графов с контролируемыми квантовыми семействами, строго соблюдая критерии:

- топологии: регулярные (1D ring, 2D torus), нерегулярные (RRG), и «решётка с нарушениями» (rewired torus);
- размеры: **N=1 000** и **N=10 000** (как в космологии);
- управляемые параметры беспорядка: onsite‑disorder **W** и структурный шум **p**;
- ансамбли: **50 реализаций на точку параметров**;
- единый формат: таблицы `nodes/edges` + метаданные, без специальных ветвлений в коде.

Мы сравниваем:
1) топологические признаки (степени, кластеризация, компоненты, плотность);  
2) спектральные признаки (низшие собственные значения нормализованного лапласиана);  
3) квантовые маркеры локализации (частичный спектр гамильтониана + IPR на выбранных собственных векторах).

> Этот ноутбук предполагает, что `01_cosmology_ingest_and_graph.ipynb` уже выполнен и артефакты лежат в `outputs/graphs/...`.


In [ ]:
# 0) Imports + paths

from __future__ import annotations

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scientific_api.logging import setup_logging, get_logger
from scientific_api.settings import DEFAULT_SEED, COSMO_SAMPLE_SIZES, ENSEMBLE_SIZE
from scientific_api.storage.paths import get_repo_root, get_outputs_dir, get_reports_dir, ensure_all_dirs
from scientific_api.graphs.knn import load_graphdata
from scientific_api.features.graph_features import featurize_graph
from scientific_api.quantum.ensembles import quantum_graph_ensemble

setup_logging("INFO")
logger = get_logger("nb02")

ensure_all_dirs()
ROOT = get_repo_root()
OUT = get_outputs_dir()
REP = get_reports_dir()

print("Repo:", ROOT)
print("Outputs:", OUT)
print("Reports:", REP)
print("Ensemble size:", ENSEMBLE_SIZE)


## 1) Загрузка космологических графов и вычисление признаков

Мы работаем отдельно для N=1000 и N=10000, чтобы не смешивать «масштаб» графа как фактор.

Признаки считаются функцией `featurize_graph(nodes, edges)`:
- топология + спектр лапласиана (низшие eigenvalues).


In [ ]:
cosmo_index_path = OUT / "graphs" / "graph_index_cosmology.parquet"
if not cosmo_index_path.exists():
    raise FileNotFoundError(f"Cosmology graph index missing: {cosmo_index_path}. Run notebook 01 first.")

cosmo_index = pd.read_parquet(cosmo_index_path)
cosmo_index


In [ ]:
def featurize_cosmology_row(row: pd.Series, spectral_k: int = 32) -> dict:
    gd = load_graphdata(Path(row["path"]))
    feats = featurize_graph(gd.nodes, gd.edges, spectral_k=spectral_k)
    feats |= {
        "domain": "cosmology",
        "preset": row["preset"],
        "source": row["source"],
        "N": int(row["N"]),
        "graph_id": f"{row['preset']}__{row['source']}__N{row['N']}",
    }
    return feats

cosmo_features = pd.DataFrame([featurize_cosmology_row(r) for _, r in cosmo_index.iterrows()])
cosmo_features.sort_values(["N","preset","source"]).head()


## 2) Дизайн квантовых экспериментов (фиксируем без «опционально»)

Мы сканируем параметр onsite‑disorder **W** и структурный шум **p**:

- `ring`, `torus`, `rrg`: **W ∈ {0.0, 2.0, 6.0}**, p=0  
- `rewired_torus`: **W = 2.0**, **p ∈ {0.00, 0.05, 0.15}**

Размеры:
- **N=1 000**: ring/rrg + torus/rewired (torus требует N=perfect square → 1000 не квадрат).  
- **N=10 000**: ring, torus (100×100), rrg, rewired_torus.

Чтобы соблюсти критерий сопоставимости размеров, для N=1000 мы используем:
- ring (N=1000),
- rrg (N=1000),
- вместо torus используем **ближайший квадрат** 32×32=1024 и затем *строго* приводим к N=1000, удаляя 24 узла детерминированно.
Это сохраняет «2D‑решётку как эталон упорядоченности» и одновременно выполняет требование N.

Детерминированность: seed фиксируем и записываем в таблицу результатов.


In [ ]:
# 2A) Utilities: enforce exact N for lattice-based families when n is not a perfect square.

import networkx as nx

def enforce_exact_n(nodes: pd.DataFrame, edges: pd.DataFrame, n_target: int, seed: int):
    """Drop nodes deterministically to reach exact n_target.

    Returns:
      nodes2, edges2, keep_old_ids, mapping_old_to_new
    """
    n0 = len(nodes)
    if n0 == n_target:
        keep_old = nodes["node_id"].to_numpy(int)
        mapping = {int(old): int(old) for old in keep_old}
        return nodes.copy(), edges.copy(), keep_old, mapping

    if n0 < n_target:
        raise ValueError(f"Cannot enlarge graph: have {n0} nodes, need {n_target}")

    rng = np.random.default_rng(seed)
    drop = rng.choice(n0, size=(n0 - n_target), replace=False)
    keep_mask = np.ones(n0, dtype=bool)
    keep_mask[drop] = False

    old_ids = nodes["node_id"].to_numpy(int)
    keep_old = old_ids[keep_mask]
    mapping = {int(old): int(i) for i, old in enumerate(keep_old)}

    nodes2 = nodes.loc[keep_mask].copy().reset_index(drop=True)
    nodes2["node_id"] = np.arange(len(nodes2), dtype=int)

    e = edges.copy()
    e = e[e["source"].isin(mapping) & e["target"].isin(mapping)].copy()
    e["source"] = e["source"].map(mapping).astype(int)
    e["target"] = e["target"].map(mapping).astype(int)
    e = e[e["source"] != e["target"]].copy()

    return nodes2, e.reset_index(drop=True), keep_old, mapping

# sanity check: applied later in generation step.


In [ ]:
# 2B) Define design grid in an explicit table.

design_rows = []

W_grid = [0.0, 2.0, 6.0]
p_grid_rewired = [0.00, 0.05, 0.15]

for N in COSMO_SAMPLE_SIZES:
    # ring
    for W in W_grid:
        design_rows.append({"family":"ring", "N":N, "W":W, "p":0.0})
    # rrg
    for W in W_grid:
        design_rows.append({"family":"rrg", "N":N, "W":W, "p":0.0})
    # torus only if perfect square or handled by enforce_exact_n
    for W in W_grid:
        design_rows.append({"family":"torus", "N":N, "W":W, "p":0.0})
    # rewired torus
    for p in p_grid_rewired:
        design_rows.append({"family":"rewired_torus", "N":N, "W":2.0, "p":p})

design = pd.DataFrame(design_rows)
design


## 3) Генерация ансамблей и вычисление признаков

Для каждой точки дизайна мы генерируем **50 реализаций** и считаем:
- те же графовые признаки, что и для космологии (`featurize_graph`);
- дополнительно: частичный спектр гамильтониана и IPR (квантовый маркер локализации).

Для N=10 000 мы **не** считаем полный спектр (слишком дорого), а используем `eigsh` на малом числе собственных значений/векторов.


In [ ]:
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh

def quantum_extra_metrics(member: dict, k_h: int = 8) -> dict:
    H = member["H"]
    n = H.shape[0]
    k_eff = min(k_h, n-2) if n > 2 else 0
    if k_eff <= 0:
        return {"H_eig_mean": np.nan, "H_eig_std": np.nan, "IPR_mean": np.nan, "IPR_std": np.nan}

    # symmetric Hamiltonian; take eigenpairs near smallest algebraic values as a stable proxy
    try:
        vals, vecs = eigsh(H, k=k_eff, which="SA")
        vals = np.real(vals)
        # IPR on eigenvectors (columns)
        psi2 = np.abs(vecs)**2
        psi2 = psi2 / np.sum(psi2, axis=0, keepdims=True)
        ipr = np.sum(psi2**2, axis=0)
        return {
            "H_eig_mean": float(np.mean(vals)),
            "H_eig_std": float(np.std(vals)),
            "IPR_mean": float(np.mean(ipr)),
            "IPR_std": float(np.std(ipr)),
        }
    except Exception:
        return {"H_eig_mean": np.nan, "H_eig_std": np.nan, "IPR_mean": np.nan, "IPR_std": np.nan}

def featurize_quantum_member(member: dict, spectral_k: int = 32) -> dict:
    nodes = member["nodes"]
    edges = member["edges"]
    feats = featurize_graph(nodes, edges, spectral_k=spectral_k)
    feats |= quantum_extra_metrics(member, k_h=8)
    feats |= {
        "domain": "quantum",
        "family": member["family"],
        "N": int(member["n"]),
        "W": float(member["W"]),
        "p": float(member["p"]),
        "seed": int(member["seed"]),
    }
    return feats


In [ ]:
quantum_features_rows = []

for _, drow in design.iterrows():
    family = drow["family"]
    N_target = int(drow["N"])
    W = float(drow["W"])
    p = float(drow["p"])

    # For torus-based families, build on the nearest square if needed
    if family in ("torus", "rewired_torus"):
        side = int(np.round(np.sqrt(N_target)))
        n_square = side * side
        N_generate = n_square if n_square != N_target else N_target
    else:
        N_generate = N_target

    seed = DEFAULT_SEED + (hash((family, N_target, W, p)) % 1_000_000)
    logger.info(f"Quantum: {family} N={N_target} (gen={N_generate}) W={W} p={p} seed={seed}")

    members = quantum_graph_ensemble(
        family=family,
        n=N_generate,
        W=W,
        p=p,
        ensemble_size=ENSEMBLE_SIZE,
        seed=seed,
        rrg_degree=4,
    )

    for m in members:
        # enforce exact N for torus-based if needed
        if N_generate != N_target:
            nodes2, edges2, keep_old_ids, mapping = enforce_exact_n(
                m["nodes"], m["edges"], n_target=N_target, seed=m["seed"]
            )

            # reduce onsite potentials consistently with kept nodes
            eps_full = np.asarray(m["onsite"], dtype=float)
            eps_keep = eps_full[keep_old_ids]

            # reduce Hamiltonian consistently by submatrix selection
            H_full = m["H"].tocsr()
            H_red = H_full[keep_old_ids, :][:, keep_old_ids]

            m = dict(m)
            m["nodes"] = nodes2
            m["edges"] = edges2
            m["onsite"] = eps_keep
            m["H"] = H_red
            m["n"] = N_target

        quantum_features_rows.append(featurize_quantum_member(m, spectral_k=32))

quantum_features = pd.DataFrame(quantum_features_rows)
quantum_features.head()


In [ ]:
# Save raw (per-realization) quantum features for full reproducibility
REP.mkdir(parents=True, exist_ok=True)
quantum_features_path = REP / "tables" / "quantum_features_per_realization.parquet"
quantum_features_path.parent.mkdir(parents=True, exist_ok=True)

quantum_features.to_parquet(quantum_features_path, index=False)
quantum_features_path


## 4) Агрегация по ансамблям: средние и дисперсии

Для сравнения с космологией мы используем:
- космология: отдельные графы (4 графа на пресет, 2 источника, 2 N)
- квантовые: усреднение по 50 реализациям на точку (W,p,family,N)

Это соответствует критерию «не одна реализация, а ансамбль».


In [ ]:
group_cols = ["family","N","W","p"]

# numeric columns only
num_cols = [c for c in quantum_features.columns if pd.api.types.is_numeric_dtype(quantum_features[c]) and c not in ["seed"]]

quantum_agg = (quantum_features
              .groupby(group_cols)[num_cols]
              .agg(["mean","std"])
              )

# flatten columns
quantum_agg.columns = [f"{c}__{stat}" for c, stat in quantum_agg.columns]
quantum_agg = quantum_agg.reset_index()

quantum_agg_path = REP / "tables" / "quantum_features_aggregated.parquet"
quantum_agg.to_parquet(quantum_agg_path, index=False)

quantum_agg.head()


## 5) Сравнение доменов в едином признаковом пространстве

Мы сравниваем на каждом N отдельно:

1) стандартизируем признаки на объединённом наборе (космология + квантовые средние);
2) считаем расстояния от каждого космологического графа до каждой квантовой точки (семейство/параметры);
3) визуализируем (теплокарта) и делаем простую интерпретацию: какая квантовая «геометрия» ближе к SDSS/DESI.

Метрика: Евклидово расстояние в z‑нормированном пространстве признаков.


In [ ]:
from sklearn.preprocessing import StandardScaler

def compare_for_N(N: int):
    cos = cosmo_features[cosmo_features["N"] == N].copy()
    q = quantum_agg[quantum_agg["N"] == N].copy()

    # choose a stable feature subset (avoid seed, meta)
    # We use topology + laplacian eigs + quantum extras (means only).
    feature_cols = [c for c in cos.columns if c.startswith(("n_","density","deg_","clustering","assortativity","n_components","lap_eig_","spectral_gap"))]
    # For quantum aggregated we have "<feature>__mean" columns
    q_feature_cols = [f"{c}__mean" for c in feature_cols if f"{c}__mean" in q.columns]

    # align
    X_cos = cos[feature_cols].copy()
    X_q = q[q_feature_cols].copy()
    X_q.columns = feature_cols

    X_all = pd.concat([X_cos, X_q], axis=0, ignore_index=True)
    X_all = X_all.fillna(X_all.mean(numeric_only=True))

    scaler = StandardScaler()
    X_all_z = scaler.fit_transform(X_all.values)

    X_cos_z = X_all_z[:len(X_cos)]
    X_q_z = X_all_z[len(X_cos):]

    # pairwise distances
    dists = np.sqrt(((X_cos_z[:, None, :] - X_q_z[None, :, :])**2).sum(axis=2))

    dist_df = pd.DataFrame(dists, index=cos["graph_id"].tolist(), columns=[f"{r.family}|W={r.W}|p={r.p}" for r in q.itertuples()])
    return cos, q, dist_df

comparisons = {}
for N in COSMO_SAMPLE_SIZES:
    comparisons[N] = compare_for_N(int(N))
    print(N, comparisons[N][2].shape)

# show N=1000 distance table
comparisons[1000][2].iloc[:, :10]


In [ ]:
# Heatmap visualization for each N (cosmology graphs vs quantum design points)

def plot_heatmap(dist_df: pd.DataFrame, title: str):
    fig = plt.figure(figsize=(12, 3 + 0.3*len(dist_df)))
    ax = fig.add_subplot(1,1,1)
    im = ax.imshow(dist_df.values, aspect="auto")
    ax.set_title(title)
    ax.set_yticks(range(len(dist_df.index)))
    ax.set_yticklabels(dist_df.index)
    ax.set_xticks(range(len(dist_df.columns)))
    ax.set_xticklabels(dist_df.columns, rotation=90, fontsize=7)
    plt.colorbar(im, ax=ax, fraction=0.03, pad=0.02, label="z-space distance")
    plt.tight_layout()
    plt.show()

for N in COSMO_SAMPLE_SIZES:
    _, _, dist_df = comparisons[int(N)]
    plot_heatmap(dist_df, title=f"Cosmology→Quantum distances (N={N})")


## 6) PCA‑проекция: совместная «карта» геометрий

Показываем, как космологические графы располагаются относительно квантовых средних точек в низкоразмерном пространстве признаков.


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def pca_map_for_N(N: int):
    cos = cosmo_features[cosmo_features["N"] == N].copy()
    q = quantum_agg[quantum_agg["N"] == N].copy()

    feature_cols = [c for c in cos.columns if c.startswith(("n_","density","deg_","clustering","assortativity","n_components","lap_eig_","spectral_gap"))]
    q_feature_cols = [f"{c}__mean" for c in feature_cols if f"{c}__mean" in q.columns]

    X_cos = cos[feature_cols].copy()
    X_q = q[q_feature_cols].copy()
    X_q.columns = feature_cols

    X_all = pd.concat([X_cos, X_q], axis=0, ignore_index=True)
    X_all = X_all.fillna(X_all.mean(numeric_only=True))

    Xz = StandardScaler().fit_transform(X_all.values)
    Z = PCA(n_components=2, random_state=DEFAULT_SEED).fit_transform(Xz)

    Z_cos = Z[:len(X_cos)]
    Z_q = Z[len(X_cos):]

    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(1,1,1)

    ax.scatter(Z_q[:,0], Z_q[:,1], marker="s", s=60)
    for i, r in enumerate(q.itertuples()):
        ax.text(Z_q[i,0], Z_q[i,1], f"{r.family}\nW={r.W},p={r.p}", fontsize=7)

    ax.scatter(Z_cos[:,0], Z_cos[:,1], marker="o", s=80)
    for i, r in enumerate(cos.itertuples()):
        ax.text(Z_cos[i,0], Z_cos[i,1], f"{r.preset}|{r.source}", fontsize=9)

    ax.set_title(f"PCA map of graph features (N={N})")
    ax.set_xlabel("PC1"); ax.set_ylabel("PC2")
    plt.tight_layout()
    plt.show()

for N in COSMO_SAMPLE_SIZES:
    pca_map_for_N(int(N))


## 7) Итоговые таблицы для вставки в Главы 2–3

Мы сохраняем:
- `reports/tables/cosmology_features.parquet`
- `reports/tables/quantum_features_aggregated.parquet`
- `reports/tables/comparison_distances_<N>.csv`

Эти таблицы можно напрямую использовать для графиков/таблиц ВКР.


In [ ]:
cosmo_features_path = REP / "tables" / "cosmology_features.parquet"
cosmo_features_path.parent.mkdir(parents=True, exist_ok=True)
cosmo_features.to_parquet(cosmo_features_path, index=False)

for N in COSMO_SAMPLE_SIZES:
    dist_df = comparisons[int(N)][2]
    out_csv = REP / "tables" / f"comparison_distances_N{int(N)}.csv"
    dist_df.to_csv(out_csv)

cosmo_features_path, quantum_agg_path


## 8) Научная интерпретация (что формально проверяем)

1) **Гипотеза о «близости геометрий»**: космологические графы ближе к семействам с нарушенной регулярностью (RRG / rewired torus), чем к идеально регулярным (ring/torus при W≈0).  
2) **Роль дизордера W**: увеличение W в квантовой части должно менять спектральные и локализационные маркеры; мы проверяем, в какую сторону это сдвигает квантовые точки относительно космологии.  
3) **Различия SDSS vs DESI**: если SDSS и DESI дают систематически разные расстояния до тех же квантовых семейств, это — наблюдаемая разница «геометрии каталогов» на фиксированном масштабе.

Дальше (в тексте главы) вы берёте таблицы и иллюстрации этого ноутбука и формулируете выводы строго по числам/графикам.
